In [1]:
# import pandas as pd
from pyspark import SparkContext
import time
# import numpy as np
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt
# import seaborn as sns
startTime = time.time()

In [2]:
sc = SparkContext("local[*]", "UserUserRating")

In [3]:
trainData = sc.textFile("user_business_rating_user_sim.csv")
trainDataFirst = trainData.first()
trainData = trainData.filter(lambda x: x != trainDataFirst)
trainData = trainData.map(lambda l: l.split(','))
trainData = trainData.map(lambda x : (x[1],x[2],x[3])).collect()

In [4]:
trainData = trainData[:40000]

In [5]:
users = {}
businesses = {}
finalInput = []
ratings = []

In [6]:
for i in trainData:
    if users.get(i[0]) is None:
        users[i[0]] = len(users) + 1
    if businesses.get(i[1]) is None:
        businesses[i[1]] = len(businesses) + 1
    ratings.append(float(i[2].strip()))
    finalInput.append([users[i[0]],businesses[i[1]],float(i[2].strip())])

In [7]:
invUsers = {str(v): k for k, v in users.items()}
invBusinesses = {str(v): k for k, v in businesses.items()}

trainAvgRating = [float(sum(col))/len(col) for col in zip(*finalInput)][2]
minVal = min(ratings)
maxVal = max(ratings)


In [8]:
finalTrainInput = sc.parallelize(finalInput)

userAverage = finalTrainInput.map(lambda l: (l[0], l[2])).mapValues(lambda l: (l,1)).reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))
userAverage = userAverage.mapValues(lambda l: 1.0*l[0]/l[1]).collectAsMap()

businesAverage = finalTrainInput.map(lambda l: (l[1], l[2])).mapValues(lambda l: (l,1)).reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))
businesAverage = businesAverage.mapValues(lambda l: 1.0*l[0]/l[1]).collectAsMap()

In [9]:
trainRdd = finalTrainInput
trainRddUsers = trainRdd.map(lambda x : ((str(x[0]), str(x[1])), (float(x[2])-minVal)/(maxVal-minVal)))
trainingRddUsers = trainRddUsers.map(lambda x : (x[0][0], (x[0][1], x[1])))
businessesOnlyUserCF = trainingRddUsers.map(lambda x : (x[1][0], x[0]))
businessesOnlyUserCF = businessesOnlyUserCF.groupByKey()
businessesOnlyUserCF = businessesOnlyUserCF.mapValues(list)
businessesOnlyUserCF = businessesOnlyUserCF.collectAsMap()
userAndBusinessesUserCF = trainingRddUsers.map(lambda x : ((x[0], x[1][0]), x[1][1]))
userAndBusinessesUserCF = userAndBusinessesUserCF.collectAsMap()

usersOnlyUserCF = trainingRddUsers.groupByKey()
usersOnlyUserCF = usersOnlyUserCF.mapValues(list)
usersOnlyUserCF = usersOnlyUserCF.collectAsMap()

In [10]:
def calcPearsonUsers(corratedUsersBusinessess):
    if len(corratedUsersBusinessess) == 0:
        return -100
    if len(corratedUsersBusinessess) == 1:
        return -100
    tempUB = [x[1][0] for x in corratedUsersBusinessess]
    tempUBAvg = sum(tempUB)/len(tempUB)
    pearsonNumerator = 0
    pearsonDenom1 = 0
    pearsonDenom2 = 0
    allUB = [x[1][1] for x in corratedUsersBusinessess]
    for (i, j) in zip(tempUB, allUB):
        pearsonNumerator += (i-tempUBAvg)*(j-tempUBAvg)
        pearsonDenom1 += (i-tempUBAvg)**2
        pearsonDenom2 += (j-tempUBAvg)**2
    pearsonDenom = (pearsonDenom1**0.5) * (pearsonDenom2**0.5)
    if pearsonDenom == 0 or pearsonNumerator == 0:
        return -100
    else:
        return pearsonNumerator/pearsonDenom

In [11]:
user_ids = usersOnlyUserCF.keys()

In [12]:
pearson_between_users= []

In [13]:
for i,user1 in enumerate(user_ids[:-1]):
    for j , user2 in enumerate(user_ids[i+1:]):
        eachUserInfo = usersOnlyUserCF[user1]
        tempKnnUsersInfo = usersOnlyUserCF[user2]
        corratedUsersBusinessess = []
        tempKnnUsersInfo.sort()
        eachUserInfo.sort()
        l= 0
        k = 0
        while l<len(tempKnnUsersInfo) and k< len(eachUserInfo):
            if tempKnnUsersInfo[l][0] == eachUserInfo[k][0]:
                corratedUsersBusinessess.append((tempKnnUsersInfo[l][0], (tempKnnUsersInfo[l][1], eachUserInfo[k][1])))
                l = l+1
                k = k+1
            elif tempKnnUsersInfo[l][0] < eachUserInfo[k][0]:
                l = l+1
            else:
                k = k+1
       
        pearsonRating = calcPearsonUsers(corratedUsersBusinessess)
        if pearsonRating != -100:
            pearson_between_users.append((user1,user2,pearsonRating))

In [14]:
pearson_between_users

[('643', '554', -0.7071067811865475),
 ('592', '2196', 0.894427190999916),
 ('345', '2440', 0.8944271909999159),
 ('345', '1997', -0.08058229640253797),
 ('345', '248', -0.8164965809277263),
 ('345', '411', -0.31622776601683794),
 ('345', '151', 0.42640143271122116),
 ('345', '1018', -0.7071067811865475),
 ('345', '2390', 0.763762615825973),
 ('345', '1114', -2.943923360032079e-16),
 ('345', '2383', -0.4472135954999579),
 ('345', '1791', 0.4029114820126903),
 ('345', '1270', -0.9999999999999998),
 ('345', '528', 0.9999999999999998),
 ('345', '1334', 0.9999999999999998),
 ('345', '2002', 1.0000000000000002),
 ('345', '2124', 0.6444259362492657),
 ('345', '124', -0.5773502691896255),
 ('346', '1384', -0.9999999999999998),
 ('342', '241', -0.24253562503633297),
 ('342', '1334', 0.894427190999916),
 ('349', '1815', 0.4472135954999579),
 ('349', '157', 0.8944271909999159),
 ('349', '1701', -0.4472135954999579),
 ('349', '823', -0.4472135954999579),
 ('349', '2404', -0.5547001962252291),
 ('

In [31]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession(sc)

In [37]:
pearson_rdd = sc.parallelize(pearson_between_users)

In [38]:

pearson_rdd = sc.parallelize(pearson_between_users)  


# convert to code and Use below to dump user-user pearson to file